In [1]:
import warnings
import pickle
import gensim
import numpy as np
from sklearn.linear_model import LogisticRegression

C:\Users\Sanjay\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
warnings.filterwarnings('ignore')

### Loading data:
***

In [3]:
with open('../data/clean-data-nostem.pkl','rb') as fp:
    X,Y = pickle.load(fp)
fp.close()

### Tokenizing:
***

In [4]:
X = [i.split() for i in X]

### Loading pre-trained model:
***

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin', binary=True)  

### Creating mean weighted sentence vector:
***

In [6]:
sen_vector = np.empty((len(X),300)); index = 0
for i in X:
    counter = 0
    vector = np.zeros(300)
    for word in i:
        counter = counter+1
        try:
            vector += model[word]
        except:
            counter = counter-1
    vector = vector/counter
    sen_vector[index] = vector
    index += 1

In [7]:
l = int(len(X)*0.8)

In [8]:
pred = {}

### Logistic Regression (L1):
***

In [9]:
lr = LogisticRegression(penalty='l1',C=1,n_jobs=-1,class_weight='balanced')
lr.fit(sen_vector[:l],Y[:l])
pred['LR L1 pre-trained W2V'] = lr.predict(sen_vector[l:])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').